In [2]:
import polars as pl
import numpy as np
import os
import altair as alt
import pysam
import itertools
alt.data_transformers.enable("vegafusion")
WINDOW_SIZE = 32

In [ ]:
bam = pysam.AlignmentFile(os.path.expanduser('~/mutationalscanning/Workspaces/chcharlton/methyl-jepa/data/raw/methylated_subset.bam'), "rb", check_sq=False)

In [29]:
def get_bam_tags(bam_path: str) -> set[str]:
    with pysam.AlignmentFile(os.path.expanduser(bam_path), "rb", check_sq=False) as bam_file:
        return {tag for read in bam_file for tag, _, in read.get_tags()}
    
def get_bam_tags(bam_path: str, n: int) -> set[str]:
    with pysam.AlignmentFile(os.path.expanduser(bam_path), "rb", check_sq=False) as bam_file:
        reads = itertools.islice(bam_file, n)
        return {tag for read in reads for tag, _,in read.get_tags()}

get_bam_tags('~/mutationalscanning/Workspaces/chcharlton/methyl-jepa/data/raw/methylated_subset.bam', n=10_000)

{'RG', 'ec', 'fi', 'fn', 'fp', 'np', 'ri', 'rn', 'rp', 'rq', 'sn', 'zm'}

In [37]:
from dataset import MethylIteratbleDataset

ModuleNotFoundError: No module named 'dataset'

In [31]:
get_bam_tags("~/mutationalscanning/DerivedData/bam/HiFi/chimp/martin/kinetics/martin_kinetics_diploid.bam", n=10_000)

{'ML',
 'MM',
 'NM',
 'RG',
 'SA',
 'ac',
 'bc',
 'bl',
 'bq',
 'bt',
 'bx',
 'cx',
 'ec',
 'fi',
 'fn',
 'fp',
 'ls',
 'ma',
 'mc',
 'mg',
 'np',
 'qe',
 'ql',
 'qs',
 'qt',
 'ri',
 'rm',
 'rn',
 'rp',
 'rq',
 'sa',
 'sm',
 'sn',
 'sx',
 'we',
 'ws',
 'zm'}

In [10]:
q = (pl.scan_parquet('../data/processed/martin_1m.parquet')
     .head()
)
q.collect()

read_name,cg_pos,seq,qual,np,sm,sx,fi,fp,ri,rp
str,u16,str,list[u8],u8,list[u8],list[u8],list[u16],list[u16],list[u16],list[u16]
"""m84108_241124_011319_s3/125309…",61,"""CATTAGACAGATCAACGAGACAGAAAGGTA…","[36, 43, … 31]",4,"[6, 6, … 6]","[0, 0, … 0]","[15, 129, … 15]","[84, 33, … 12]","[77, 157, … 10]","[11, 13, … 11]"
"""m84108_241124_011319_s3/125309…",326,"""AAACTCACTCAAAACCGCTCAACTACATGG…","[31, 47, … 60]",4,"[6, 6, … 6]","[0, 0, … 0]","[63, 60, … 57]","[11, 29, … 14]","[28, 60, … 40]","[5, 20, … 12]"
"""m84108_241124_011319_s3/125309…",797,"""AGGGGATATCACCACCGATCCCACAGAAAT…","[27, 57, … 36]",4,"[6, 6, … 6]","[0, 0, … 0]","[28, 130, … 56]","[46, 7, … 22]","[30, 53, … 22]","[42, 30, … 23]"
"""m84108_241124_011319_s3/125309…",848,"""CTATAAACACCTCTACGCAAATAAACTAGA…","[34, 47, … 63]",4,"[6, 6, … 6]","[0, 0, … 0]","[82, 93, … 65]","[21, 12, … 11]","[134, 113, … 74]","[32, 15, … 31]"
"""m84108_241124_011319_s3/125309…",1335,"""ACACAAATCAATAAACGTAATCCAGCATAT…","[35, 35, … 59]",4,"[5, 5, … 6]","[1, 0, … 0]","[11, 26, … 18]","[7, 22, … 8]","[37, 161, … 16]","[16, 38, … 17]"


In [ ]:
q = (pl.scan_parquet('../data/processed/null/martin_null_p0.024_n1m.parquet')
)
df = q.collect()
df.select('read_name').n_unique()

22

In [2]:
subset_q = (
    pl.scan_parquet(os.path.expanduser('~/mutationalscanning/Workspaces/chcharlton/methyl-jepa/data/processed/standard_600k_32_train.parquet'),
                    schema = {'read_name': pl.String,
                              'cg_pos': pl.Int64,
                              'seq': pl.String,
                              'fi': pl.List(pl.UInt16),
                              'fp': pl.List(pl.UInt16),
                              'ri': pl.List(pl.UInt16),
                              'rp': pl.List(pl.UInt16),
                              'label': pl.Int32
                              })
    .head(1_000_000)
    )

KINETICS_FEATURES = ['fi', 'ri', 'fp', 'rp']
subset_df = subset_q.collect()

def compute_log_normalization_stats(df, features, epsilon=1):
    means = {col: (df[col].explode() + epsilon).log().mean() for col in features}
    stds = {col: (df[col].explode() + epsilon).log().explode().std() for col in features}
    return means, stds

train_means, train_stds = compute_log_normalization_stats(subset_df, KINETICS_FEATURES)
print(train_means, train_stds)


{'fi': 3.2361628985125575, 'ri': 3.2246924771219243, 'fp': 3.0442874824956117, 'rp': 3.0464773632625053} {'fi': 0.6563387717253599, 'ri': 0.6509892180172931, 'fp': 0.44864215501330723, 'rp': 0.4492703347296856}


In [4]:
subset_q = (
    pl.scan_parquet(os.path.expanduser('~/mutationalscanning/Workspaces/chcharlton/methyl-jepa/data/processed/null/martin_null_p0.24_n1m.parquet'))
    .head(1_000_000)
    )

KINETICS_FEATURES = ['fi', 'ri', 'fp', 'rp']
subset_df = subset_q.collect()

def compute_log_normalization_stats(df, features, epsilon=1):
    means = {col: (df[col].explode() + epsilon).log().mean() for col in features}
    stds = {col: (df[col].explode() + epsilon).log().explode().std() for col in features}
    return means, stds

train_means, train_stds = compute_log_normalization_stats(subset_df, KINETICS_FEATURES)
print(train_means, train_stds)


{'fi': 2.9962848372251614, 'ri': 3.0480021741148122, 'fp': 2.848003419153076, 'rp': 2.8412808948316832} {'fi': 0.674978172935705, 'ri': 0.7026555651455643, 'fp': 0.48432900536167434, 'rp': 0.4824820324841106}


In [33]:
q = (pl.scan_parquet(os.path.expanduser('~/mutationalscanning/Workspaces/chcharlton/methyl-jepa/data/processed/standard_600k_32_v2_test_test.parquet'))
     .head()
)
q.collect()

read_name,cg_pos,seq,qual,np,fi,fp,ri,rp,label
str,i64,str,list[u8],u8,list[u16],list[u16],list[u16],list[u16],i32
"""m64168_200823_191315/237/ccs""",5286,"""GGGTATAAAAAGTTGCGTATTTTCAGTGAA…","[93, 93, … 93]",26,"[29, 20, … 23]","[20, 32, … 19]","[18, 15, … 12]","[23, 16, … 37]",1
"""m64168_200823_191315/640/ccs""",4548,"""ACCATCCCAGGTGTCCGATGCGGATGAAGA…","[93, 40, … 93]",13,"[23, 25, … 18]","[13, 30, … 22]","[6, 18, … 8]","[26, 15, … 14]",1
"""m64168_200820_000733/82/ccs""",5489,"""ATGGGGGAAGAATAGCGGTGGAAAGGCATT…","[93, 93, … 93]",13,"[14, 23, … 31]","[21, 17, … 27]","[22, 14, … 12]","[15, 13, … 11]",0
"""m64168_200823_191315/351/ccs""",1164,"""ACAACACTAAAGGTTCGCACATGTAATAGC…","[93, 61, … 63]",8,"[17, 29, … 29]","[11, 13, … 9]","[29, 73, … 11]","[21, 10, … 38]",1
"""m64168_200820_000733/112/ccs""",3665,"""CTGCAAAATCAGACTCGGAATATATTCTCT…","[93, 93, … 93]",14,"[23, 13, … 22]","[18, 13, … 18]","[12, 12, … 15]","[17, 19, … 23]",0


In [4]:
q = (pl.scan_parquet(os.path.expanduser('~/mutationalscanning/Workspaces/chcharlton/methyl-jepa/data/processed/pacbio_standard_train'))
     .head()
)
q.collect()

read_name,cg_pos,seq,qual,np,fi,fp,ri,rp,label
str,i64,str,list[u8],u8,list[u16],list[u16],list[u16],list[u16],i32
"""m64168_200820_000733/29/ccs""",1661,"""AAAACAGTCTGTAGCCGGGCGTGGTGGCTC…","[79, 79, … 49]",3,"[52, 35, … 59]","[12, 7, … 13]","[69, 38, … 70]","[16, 61, … 14]",0
"""m64168_200820_000733/29/ccs""",1731,"""GATTGCCTGAGCTCACGAGTTCAAGACCAG…","[61, 53, … 76]",3,"[64, 71, … 38]","[42, 40, … 18]","[72, 107, … 60]","[8, 11, … 12]",0
"""m64168_200823_191315/30/ccs""",6596,"""TCAGAAACAGCTCCTCGTGGTCCAGTCACC…","[93, 93, … 93]",30,"[14, 23, … 8]","[9, 47, … 16]","[26, 25, … 23]","[28, 25, … 24]",1
"""m64168_200823_191315/170/ccs""",4243,"""ATCAGAGCTTTCACTCGGAGTCCCTACTGG…","[93, 93, … 93]",25,"[18, 10, … 52]","[27, 9, … 27]","[21, 17, … 22]","[18, 27, … 12]",1
"""m64168_200820_000733/9/ccs""",6390,"""GGTGGTCCGTGTGGCCGTCTGCTTGGTAGC…","[46, 93, … 53]",7,"[23, 15, … 27]","[41, 25, … 16]","[47, 27, … 20]","[30, 21, … 15]",0


In [2]:
df_train = pl.read_parquet('../data/processed/standard_10k_32_test_train.parquet')
df_test = pl.read_parquet('../data/processed/standard_10k_32_test_test.parquet')
len(df_train)/(len(df_test)+len(df_train))

FileNotFoundError: No such file or directory (os error 2): ../data/processed/standard_10k_32_test_train.parquet

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


In [5]:
df_train = pl.read_parquet('../data/processed/standard_all_32_train.parquet')
# df_test = pl.read_parquet('../data/processed/standard_all_32_test.parquet')
len(df_train)/(len(df_test)+len(df_train))

: 

In [3]:
df_train['label'].mean(), df_test['label'].mean()

(0.5011783294006328, 0.5069963935528348)

In [4]:
df_train.head()

read_name,cg_pos,seq,fi,fp,ri,rp,label
str,i64,str,list[u16],list[u16],list[u16],list[u16],i32
"""m64168_200823_191315/2231102/c…",1984,"""GAGCCGGGGTTTCACCGCGTTAGCCAGGAT…","[21, 27, … 14]","[14, 21, … 46]","[18, 33, … 40]","[19, 14, … 31]",1
"""m64168_200820_000733/262810/cc…",6481,"""TATCTAAATGAAGTACGTATTTATACAAGT…","[14, 20, … 38]","[27, 22, … 22]","[33, 22, … 17]","[38, 12, … 14]",0
"""m64168_200823_191315/1180098/c…",7092,"""TAACATGCTATAACACGCTGCACATGCAAC…","[27, 37, … 20]","[37, 23, … 33]","[25, 27, … 10]","[15, 8, … 11]",1
"""m64168_200820_000733/461584/cc…",6056,"""AGGATCACTTGAGCTCGGGAAGCAGAGGCT…","[34, 23, … 28]","[20, 23, … 30]","[76, 24, … 21]","[33, 18, … 11]",0
"""m64168_200820_000733/330109/cc…",1103,"""TGCAGCACTAGCAGACGCGCCATGTGTGTT…","[34, 23, … 11]","[10, 18, … 41]","[28, 20, … 17]","[18, 7, … 8]",0


In [6]:
df_train['fi'].to_numpy()

array([array([52, 29, 39, 35, 20, 23,  6, 23, 34, 40, 38, 35, 18, 69, 22, 35, 21,
              22, 18, 15, 29, 22, 27, 66, 62, 45, 23, 37, 11,  9,  8, 21])       ,
       array([ 19,  60,  29,  50,  45,  29,  35,  14,  14,  59,  39,  20,  81,
               56,  33,  52,  21,  22,  50,  23, 126,  70,  63,  25, 145,  43,
               55,  20,  48,  26,  24,  39])                                  ,
       array([ 23,  59,  14,  21,  47,  27,  51,  13,  16,  69,  31,   9,  18,
               20,  45,  84,  61, 131, 163,  31,  34,  34,  70,  47,  29,  29,
               39,  32,  10,  24,  70,  69])                                  ,
       ...,
       array([11, 28, 12, 19,  9, 14, 19, 13, 21, 21, 14, 15, 24, 15, 22, 20, 30,
              16, 18, 19, 13, 12, 24, 20, 21, 23, 11, 13, 12, 19, 67, 16])       ,
       array([16, 25, 22, 13, 17, 13, 14,  8, 14, 13, 27,  9, 14, 15, 15, 13, 35,
              18, 11, 15, 11, 17, 22, 14, 15, 23, 11, 14, 14, 14, 12, 14])       ,
       array([ 26

In [ ]:
kinetic_cols = ['fi', 'fp', 'ri', 'rp']
data_array = np.stack([df_train.slice(0,1)[c].to_numpy() for c in kinetic_cols], axis=1)
data_array.shape

(1, 4)

In [ ]:
lengths = df_train["fi"].list.len()
all_equal = lengths.n_unique() == 1
print(all_equal)

True


In [ ]:
df_train = pl.read_parquet('../data/processed/train_cg_32_10000.parquet')
df_train = df_train.with_columns([
    # pl.col("seq").list.to_array(WINDOW_SIZE),  
    pl.col("fi").list.to_array(WINDOW_SIZE),  
    pl.col("fp").list.to_array(WINDOW_SIZE), 
    pl.col("ri").list.to_array(WINDOW_SIZE), 
    pl.col("rp").list.to_array(WINDOW_SIZE), 
    ])


In [ ]:

arr = np.stack([df_train[col].to_numpy() for col in kinetic_cols], axis=1)
arr.shape

(1361342, 4, 32)

In [ ]:
unique_chars = (
    df_train.select(
        pl.col("seq").str.explode().unique().alias("unique_chars")
    )
)
unique_chars

/var/folders/wt/33m104j54lb3xnbgck84qsw40000gp/T/ipykernel_1306/1930145770.py:3: DeprecationWarning: `str.explode` is deprecated; use `str.split("").explode()` instead. Note that empty strings will result in null instead of being preserved. To get the exact same behavior, split first and then use a `pl.when...then...otherwise` expression to handle the empty list before exploding.
  pl.col("seq").str.explode().unique().alias("unique_chars")


unique_chars
str
"""C"""
"""T"""
"""A"""
"""G"""


In [ ]:
vocab = {'A':0, 'T':1, 'C':2, 'G':3}
seq_ints = (
    df_train["seq"]
    .str.split("")                # Split into characters
    .list.eval(                   # Evaluate an expression on each list element
        pl.element().replace_strict(vocab)
    )
    .to_numpy()                   # Convert to NumPy array
)
np.stack(seq_ints).shape

(1361342, 32)

In [ ]:
np.stack([df_train[:5][c].to_numpy() for c in kinetic_cols], axis=1)

array([[array([52, 29, 39, 35, 20, 23,  6, 23, 34, 40, 38, 35, 18, 69, 22, 35, 21,
               22, 18, 15, 29, 22, 27, 66, 62, 45, 23, 37, 11,  9,  8, 21])       ,
        array([19, 10, 31, 16, 26, 12, 13, 23, 21,  9, 19,  8, 26, 17, 16, 22, 19,
               12, 38, 27, 19, 30, 41, 13, 14, 12, 12, 12, 17, 38, 13, 14])       ,
        array([ 62,  14,  26,  15,  71,  30,  30,  19,  12,  18,  24,  23,  36,
               139, 176,  49,  50,  49,  26,  34,  33,  27,  23,  26,  37,  38,
                32,  22,  59,  28,  43,  12])                                  ,
        array([29, 25, 19, 26, 16, 16,  7, 32, 39, 12, 33, 11, 10, 12, 28,  7, 14,
               16, 20, 24, 34, 16, 11, 15, 10, 14, 28, 22, 27, 24, 62, 13])       ],
       [array([ 19,  60,  29,  50,  45,  29,  35,  14,  14,  59,  39,  20,  81,
                56,  33,  52,  21,  22,  50,  23, 126,  70,  63,  25, 145,  43,
                55,  20,  48,  26,  24,  39])                                  ,
        array([1

In [ ]:
list_of_2d_arrays: np.stack([np.array(df_train[c].to_list()) for c in kinetic_cols], axis=1)

In [ ]:
list_of_2d_arrays

[array([[52, 29, 39, ...,  9,  8, 21],
        [19, 60, 29, ..., 26, 24, 39],
        [23, 59, 14, ..., 24, 70, 69],
        ...,
        [11, 28, 12, ..., 19, 67, 16],
        [16, 25, 22, ..., 14, 12, 14],
        [26, 91, 17, ..., 57, 21, 50]], shape=(1361342, 32)),
 array([[19, 10, 31, ..., 38, 13, 14],
        [12, 49, 15, ...,  9, 11, 24],
        [12, 25, 16, ..., 35, 12,  9],
        ...,
        [12, 17, 21, ..., 32, 24, 28],
        [14, 16, 52, ..., 29, 12, 58],
        [22, 39, 17, ..., 25,  7, 22]], shape=(1361342, 32)),
 array([[62, 14, 26, ..., 28, 43, 12],
        [31, 52, 26, ..., 34, 20, 28],
        [35, 48,  5, ..., 31, 65, 68],
        ...,
        [16, 17, 16, ..., 31, 10, 24],
        [17, 13,  6, ..., 17,  9, 19],
        [ 9, 11, 15, ..., 13, 25,  5]], shape=(1361342, 32)),
 array([[29, 25, 19, ..., 24, 62, 13],
        [12, 17, 20, ..., 24, 28, 54],
        [18, 11, 11, ..., 19,  7, 28],
        ...,
        [12, 18, 28, ..., 27, 23, 29],
        [19, 39, 45, 

In [ ]:
df_train = pl.read_parquet('../data/processed/test_run_train.parquet')
df_test = pl.read_parquet('../data/processed/test_run_test.parquet')
len(df_train)/(len(df_test)+len(df_train))

0.7999774431850223

In [ ]:
df_test['label'].mean()

0.4990132506343389

In [ ]:
import re
seq = 'ATCGATCGAA'
cgs = re.finditer('CG', seq)
print(seq)
for match in cgs:
    print(match.start())

ATCGATCGAA
2
6


In [ ]:
df_train_singletons = pl.read_parquet('../data/processed/1k_singletons_train.parquet')
df_test_singletons = pl.read_parquet('../data/processed/1k_singletons_test.parquet')
len(df_train_singletons)


85155

In [1]:
import torch

: 

In [4]:
x = torch.ones((10,20,30))
x.shape
x.permute(2,1,0).shape

torch.Size([30, 20, 10])

In [2]:
import torch

In [1]:
from src.dataset_class import MethylIterableDataset

In [ ]:
train_